# Running Your PyTorch Projects on RunPod Using a Single Docker Image and GHCR

This guide explains how to prepare your machine-learning projects for RunPod GPU compute using:

* One repository: `PyTorchTutorial`
* Multiple projects stored under:
  `/home/behnam/workspace/PyTorchTutorial/projects/`
* One Dockerfile
* One `requirements.txt`
* GitHub Container Registry (GHCR)
* Automatic dataset download via `kagglehub`

The result is a clean, scalable setup where **one Docker image** can run **any** of your Kaggle and PyTorch projects on RunPod.

---

## 1. Directory structure

The repository is:

```
/home/$USERNAME/workspace/PyTorchTutorial/
```

Inside it, the relevant content for RunPod is only:

```
PyTorchTutorial/
└── projects/
      ├── BrainCancer-MRI/
      ├── brain-mri/
      ├── CIFAR10/
      ├── Lung_Disease_Dataset/
      ├── MNIST/
      ├── segmentation/
      └── visual_odometry/
```

Each project has its own:

* `train.py`
* dataset loaders
* configuration files

RunPod will use only this `/projects` directory.

---

## 2. Create a single `requirements.txt`

This file contains Python dependencies used by **all** projects.

Create:


[/home/$USERNAME/workspace/PyTorchTutorial/requirements.txt](../environment.yml)



This ensures:

* kagglehub works inside the container
* all ML/vision utilities are available
* all projects can run without installing extra packages

---

## 3. Create a single Dockerfile

Create:

[/home/$USERNAME/workspace/PyTorchTutorial/Dockerfile](../Dockerfile)

---

## 4. Build the Docker image locally

Navigate to your repo root:

```bash
cd /home/$USERNAME/workspace/PyTorchTutorial/
```

Build the Docker image:

```bash
docker build -t ghcr.io/behnamasadi/kaggle-projects:latest .
```


Inside the container, check the size of the projects directory:

```
cd /workspace/projects
du -sh .  # Human-readable size of the current directory
```



You now have a local image ready for upload.






---

## 5. Login to GitHub Container Registry (GHCR)

Generate a GitHub PAT (Personal Access Token) with:

* read:packages
* write:packages

Then login safely:

```bash
echo YOUR_GITHUB_TOKEN | docker login ghcr.io -u behnamasadi --password-stdin
```

This avoids insecure plain-text warnings.

---

## 6. Push the image to GHCR

Execute:

```bash
docker push ghcr.io/behnamasadi/kaggle-projects:latest
```

Your image is now available from anywhere (including RunPod).

---

## 7. RunPod Setup

Go to:

RunPod Dashboard → GPU Cloud → Deploy Pod

Choose:

* Container image:
  `ghcr.io/behnamasadi/kaggle-projects:latest`
* GPU type: RTX 4090 / RTX A6000 / A100 (depending on project)
* Volume: optional (not required because kagglehub downloads automatically)


set the Environment Variables:

 
| Parameter | Value |
| :--- | :--- |
| **KAGGLE\_USERNAME** | asadibehnam |
| **KAGGLE\_KEY** | xxxxxx |
| **WANDB\_API\_KEY** | xxxxxxxx |
| **HOME** | /workspace |

 



Launch the pod.

---

## 8. Inside the RunPod terminal

When the pod is running:

Open → Web Terminal

You will see the Docker working directory:

```
/workspace/projects/
```

You can now run any project.

---

## 9. Running individual projects

#### Example: Lung Disease Dataset

```bash
cd /workspace/projects/Lung_Disease_Dataset
python train.py
```

#### Example: BrainCancer MRI

```bash
cd /workspace/projects/BrainCancer-MRI
python train.py
```

#### Example: CIFAR10

```bash
cd /workspace/projects/CIFAR10
python train.py
```

Each project works independently using the same container.

---

## 10. Using kagglehub for automatic dataset download

In each project's `train.py`, you can download datasets like:

```python
import kagglehub
import os

dataset_path = kagglehub.dataset_download("khaleddev/lungs-disease-dataset-broken")
print("Dataset stored at:", dataset_path)
```

This will store data in:

```
/root/.cache/kagglehub/datasets/<dataset-name>/versions/<version>/
```

Notes:

* First run downloads the dataset.
* Subsequent runs reuse cached versions.
* No need to upload datasets to RunPod or Docker.

---

## 11. Optional: project-level dataset symlink

If you want a clean interface inside each project:

```python
os.makedirs("data", exist_ok=True)

local_path = "data/raw"

if not os.path.exists(local_path):
    os.symlink(dataset_path, local_path)
```

Then your dataloader can always use:

```
data/raw/
```

No matter which Kaggle dataset you download.

---

## 12. Logging with TensorBoard or W&B

TensorBoard:

```bash
tensorboard --logdir logs --port 6006
```

Then expose port 6006 in RunPod settings.

Weights & Biases:

* Works automatically if you `wandb login`.

---


## 13. Download Artifacts from wandb

```python
import wandb
run = wandb.init()
artifact = run.use_artifact('behnamasadi/lung-disease-classification/run-ftbild81-last_model.pth:v0', type='unspecified')
artifact_dir = artifact.download()
```

## 14. **Clean all Docker containers and images**
Here are the commands to **clean all Docker containers and images** safely and completely.

#### 1. Remove **all containers**

First stop all running containers:

```bash
docker stop $(docker ps -aq)
```

Then remove all containers:

```bash
docker rm $(docker ps -aq)
```

#### 2. Remove **all images**

```bash
docker rmi $(docker images -aq)
```

#### 3. Remove **all volumes**

```bash
docker volume rm $(docker volume ls -q)
```



#### 5. Clean everything with a single prune command

This removes **containers, images, networks, build cache** etc.

```bash
docker system prune -a --volumes
```

This is the most complete cleanup.

---


## 15. RunPod CLI

#### Download and install via wget
```bash
wget -qO- cli.runpod.net | sudo bash
```


#### Create a Key


1. Go to [https://www.runpod.io/console/user/settings](https://www.runpod.io/console/user/settings)
2. Create an API key
3. Paste it in the command above

Login:

```bash
runpodctl config --apiKey rpa_xxxxxxxxxxxxxxxxxxxx
```

will give you:

```bash
Configuration saved to file: /home/behnam/.runpod/config.toml
```
---

#### **Download files from your pod**


```bash
runpodctl get pod
ID            	NAME                    	GPU        	IMAGE NAME                                	STATUS  
ftwh601f8kb14e	conceptual_blush_chicken	1 RTX A6000	ghcr.io/behnamasadi/kaggle-projects:latest	RUNNING	
```

you have the pod ID:

```
ftwh601f8kb14e
```

And you want to download everything inside:

```
/workspace/projects/Lung_Disease_Dataset/checkpoints
```

You can download **the whole folder** or **specific files**.

**Important:** `runpodctl receive` **does NOT take a pod-path directly.**
It requires a **“transfer code”** that is generated **inside the pod** using `runpodctl send`.



So here is the correct workflow:

#### ✅ Step 1 — SSH into your pod

```
ssh <your_ssh_user>@ssh.runpod.io
```

Your prompt should now be inside the pod:

```
pod-user@<pod>:/workspace$
```

#### ✅ Step 2 — Inside the pod, generate a “send code”

This will create a code that allows your **local machine** to download the file:

```
runpodctl send /workspace/projects/Lung_Disease_Dataset/checkpoints/best_model.pth
```

It will output something like:

```
runpodctl-receive: your code is:
abc123-def456-ghi789
```

Copy that code.

#### ✅ Step 3 — On your local machine, download using that code

On your laptop:

```
runpodctl receive abc123-def456-ghi789=./best_model.pth
```

This **will download the file**.

---

#### 📌 To download a whole folder

Inside the pod:

```
cd /workspace/projects/Lung_Disease_Dataset
runpodctl send checkpoints
```

It will output a code like:

```
runpodctl-receive: your code is:
xyz111-uvw222-rst333
```

On your local machine:

```
runpodctl receive xyz111-uvw222-rst333=./checkpoints
```

It will download the entire folder recursively.

---


